**mandatoryPropertyMissing.ipynb**

EDAM concept is missing a mandatory annotation propery

**Documentation:** https://github.com/edamontology/edamverify/blob/master/docs/mandatoryPropertyMissing.md

Set constants for script return values. Load EDAM_dev.owl from GitHub into an RDF graph.

In [ ]:
import sys
from rdflib import ConjunctiveGraph, Namespace

# Constants for script return value as per https://github.com/edamontology/edamverify.
NOERR = 0
INFO  = 1
WARN  = 2
ERROR = 3

#Load EDAM_dev.owl from GitHub into an RDF graph.
print("Loading graph ...", end="")
g = ConjunctiveGraph()
g.load('https://raw.githubusercontent.com/edamontology/edamontology/master/EDAM_dev.owl', format='xml')
# g.load('EDAM_dev.owl', format='xml')
g.bind('edam', Namespace('http://edamontology.org#'))
print("done!")



Define SPARQL query to retrieve ID, term, and all other properties that are mandatory for Operation, Data, Identifier, Format or Topic concepts. Run the query.

**NB.1:** BASE is used to define the define the default namespace (for various elements below).

**NB.2:** A concept is considered to be e.g. an Identifier if it has a property ``<oboInOwl:inSubset>`` whose value contains the string e.g. ``identifiers``, e.g. ``<oboInOwl:inSubset rdf:resource="http://purl.obolibrary.org/obo/edam#identifiers"/>`` or ``<oboInOwl:inSubset>identifiers />``

In [ ]:
# Compile SPARQL query
query_term = """
BASE <http://edamontology.org/>
SELECT ?id ?term ?subsetpath ?definition ?created_in WHERE
{
?id rdfs:label ?term .
?id oboInOwl:inSubset ?subsetpath .
OPTIONAL {?id oboInOwl:hasDefinition ?definition .}
OPTIONAL {?id :created_in ?created_in .}
}
"""

# Run SPARQL query and collate results
err = NOERR
errfound = False
report = list()
results = g.query(query_term)

Analyse results of query.

In [ ]:
# Error message building function
def report_append(err_new, msg):
    """
    Writes a report of an error and sets the error level appropriately.
    err: the error level to set
    msg: the message to write
    """
    global err
    global errfound
    global report
    if err_new > err:
        err = err_new
    errfound = True
    report.append(msg)

for r in results :
#    print(str(r['id']), str(r['term']), str(r['ext']), str(r['exact_syn']))
    id   = str(r['id'])
    term = str(r['term'])
    subsetpath  = str(r['subsetpath'])
    definition = str(r['definition'])
    created_in = str(r['created_in'])


    # Check for Format concepts
    if "operations" in subsetpath \
        or "data" in subsetpath \
        or "identifiers" in subsetpath \
        or "formats" in subsetpath \
        or "topics" in subsetpath:

        if definition == "None":
            report_append(ERROR, "ERROR Missing property (definition) on ::: " + id +  ' (' + term + ')')
        if created_in == "None":
            report_append(ERROR, "ERROR Missing property (created_in) on ::: " + id +  ' (' + term + ')')

Write report and return approriate value.

In [ ]:
# Return exit code (raises exception)
if errfound:
    print('Mandatory properties missing from these concepts:')
    print("\n".join(report))
    sys.exit(err)
else:
    print("No issues found.")
    sys.exit(NOERR)